In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests as rq
import re
import datetime
import os
import time

import plotly.offline as pyo 
import plotly.graph_objects as go
from plotly.subplots import make_subplots

color_set = {
        "dark_red"       : "#6f1926",
        "red"            : "#de324c",
        "orange"         : "#f4895f",
        "yellow"         : "#f8e16f",
        "bright_green"   : "#8bc34a",
        "dark_green"     : "#009688", #"#318a01",
        "blue"           : "#369acc",
        "sky_blue"       : "#80d9ff",
        "dark_blue"      : "#0057e5",
        "violet"         : "#9656a2",
        "purple"         : "#7b4fff",
        "black"          : "#212020",
        "gray"           : "#777675"
    }

In [2]:
stock_excel_filepath = '../Output/RawReports-05-ScrapeSummerizeStockSet/SummaryReport-20240906-081114.xlsx'
stock_excel_filepath = '../Output/RawReports-05-ScrapeSummerizeStockSet/SummaryReport-20240906-084343.xlsx'
stock_excel_filepath = '../Output/RawReports-05-ScrapeSummerizeStockSet/SummaryReport-20240906-084944.xlsx'
stock_excel_filepath = '../Output/RawReports-05-ScrapeSummerizeStockSet/SummaryReport-20240907-095717.xlsx'

In [3]:
xls = pd.ExcelFile(stock_excel_filepath)

df_basic_info               = pd.read_excel(xls, "Basic Info") # Top Ratios
df_top_ratios               = pd.read_excel(xls, "Top Ratios") # Top Ratios
df_quaterly_results         = pd.read_excel(xls, "Quarterly Results") # Quarterly Results
df_profit_n_loss            = pd.read_excel(xls, "Profit & Loss") # Profit & Loss
df_compounded_sales_growth  = pd.read_excel(xls, "Compounded Sales Growth") # Compounded Sales Growth
df_compounded_profit_growth = pd.read_excel(xls, "Compounded Profit Growth") # Compounded Profit Growth
df_stock_price_cagr         = pd.read_excel(xls, "Stock Price CAGR") # Stock Price CAGR
df_return_on_equity         = pd.read_excel(xls, "Return on Equity") # Return on Equity
df_balance_sheet            = pd.read_excel(xls, "Balance Sheet") # Balance Sheet
df_cash_flows               = pd.read_excel(xls, "Cash Flows") # Cash Flows
df_ratios                   = pd.read_excel(xls, "Ratios") # Ratios
df_shareholding_pattern     = pd.read_excel(xls, "Shareholding Pattern") # Shareholding Pattern

In [14]:
df_basic_info

,Stockname,Stock Report Type,Stock Url,Sector,Industry
0,UNITDSPR,consolidated,https://www.screener.in/company/UNITDSPR/conso...,Null,Null
1,UBL,consolidated,https://www.screener.in/company/UBL/consolidated/,Null,Null
2,RADICO,standalone,https://www.screener.in/company/RADICO/,Null,Null
3,SULA,standalone,https://www.screener.in/company/SULA/,Null,Null


In [ ]:
<table>
  <tr>
    <th>Stock Symbol</th>
    <th>Link</th>
  </tr>
  <tr>
    <td>Alfreds Futterkiste</td>
    <td>Maria Anders</td>
  </tr>
  <tr>
    <td>Centro comercial Moctezuma</td>
    <td>Francisco Chang</td>
    <td>Mexico</td>
  </tr>
</table>




In [23]:
stock_info = "<table style='padding: 15px; border: 1px solid black; font-family: Arial;'><tr><th>Stock Symbol</th><th>Link</th></tr>"
for idx, row in df_basic_info.iterrows():
    stock_name = row['Stockname']
    stock_url = row['Stock Url']
    temp = f"<tr><td>{stock_name}</td><td><a href='{stock_url}' target='_blank'>Go to Screener.in - {stock_name}</a></td></tr>"
    stock_info = stock_info + temp
    
stock_info = stock_info + "</table><br /><hr /><br />"

print(stock_info)


<table style='padding: 15px; border: 1px solid black; font-family: Arial;'><tr><th>Stock Symbol</th><th>Link</th></tr><tr><td>UNITDSPR</td><td><a href='https://www.screener.in/company/UNITDSPR/consolidated/' target='_blank'>Go to Screener.in - UNITDSPR</a></td></tr><tr><td>UBL</td><td><a href='https://www.screener.in/company/UBL/consolidated/' target='_blank'>Go to Screener.in - UBL</a></td></tr><tr><td>RADICO</td><td><a href='https://www.screener.in/company/RADICO/' target='_blank'>Go to Screener.in - RADICO</a></td></tr><tr><td>SULA</td><td><a href='https://www.screener.in/company/SULA/' target='_blank'>Go to Screener.in - SULA</a></td></tr></table><br /><hr /><br />
